In [1]:
###################################################
#####NO NEED TO MODIFY THE CODE IN THIS BLOCK######
###################################################

###IMPORT LIBRARIES###

#Numpy, array library
import numpy as np

#Matplotlib, plotting library
import matplotlib.pyplot as plt

#import os, a folder management library
import os
import shutil

#time library
import time

#makes numerical grid for landscapes
from landlab import RasterModelGrid #sets up grid

###Import the components that we need from landlab
from landlab.components import FlowAccumulator #flow accumulation
from landlab.components import FastscapeEroder #fluvial erosion
from landlab.components import LinearDiffuser #hillslope diffusion
from landlab.components import Lithology #Lithology for changing bedrock
from landlab.components import ChannelProfiler #library for plotting profile

###DEFINE FUNCTIONS###

def simple_plot(fignum, filename, figsize, title, grid, field, color_label,cmap,log_scale,v):
    plt.figure(fignum, figsize = figsize)
    data = np.flipud(grid.at_node[field].reshape(grid.shape)).copy()
    if log_scale == True:
        data[data==0]=np.nan
        im =plt.imshow(np.log10(data),cmap = cmap,extent=[0,grid.dx*grid.shape[1],0,grid.dx*grid.shape[0]],vmin=np.log10(v[0]),vmax=np.log10(v[1]))
    else:    
        im =plt.imshow(data,cmap = cmap,extent=[0,grid.dx*grid.shape[1],0,grid.dx*grid.shape[0]],vmin=v[0],vmax=v[1])
    plt.title(title)
    plt.xlabel('x [meters]')
    plt.ylabel('y [meters]')
    cbar = plt.colorbar(im,label=color_label)
    plt.tight_layout()
    if filename != "":
        plt.savefig(filename,dpi=250)
        plt.close()
    
def reset_landscape_random(grid):
    grid.at_node['topographic__elevation'] = np.random.rand(len(grid.at_node['topographic__elevation']))
    
def profile_plot(fignum, filename, figsize, title, grid, lith,v):
    profiler = ChannelProfiler(grid,number_of_watersheds=1,main_channel_only=True)
    profiler.run_one_step()
    
    plt.figure(fignum, figsize = figsize)
    outlets = list(profiler.data_structure.keys())
    distance = np.array([])
    eta = np.array([])
    for outlet in outlets:
        segments = list(profiler.data_structure[outlet].keys())
        for i, segment in enumerate(segments):
            ids = profiler.data_structure[outlet][segment]['ids']
            distance = np.concatenate((distance, profiler.data_structure[outlet][segment]['distances']), axis=None)
            eta = np.concatenate((eta, grid.at_node['topographic__elevation'][ids]), axis=None)
    
    num_layers = lith.z_top.shape[0]
    layers = np.zeros((num_layers-1,len(eta)))
    index = 0
    for outlet in outlets:
        segments = list(profiler.data_structure[outlet].keys())
        for i, segment in enumerate(segments):
            ids = profiler.data_structure[outlet][segment]['ids']
            for j in ids:
                for k in range(0,num_layers-1):
                    layers[k,index] = lith.z_top[k,j]
                index+=1
    
    for k in range(0,num_layers-1):
        plt.plot(distance,eta-layers[k,:])
        
    plt.plot(distance,eta,color='k')
    plt.title(title)
    plt.xlabel('s [meters]')
    plt.ylabel(r'$\eta$ [meters]')
    if v[0] == -9999. and v[1] != -9999.:
        plt.ylim(0,v[1])
    elif v[0] != -9999. and v[1] == -9999.:
        plt.ylim(v[0],)
    elif v[0] != -9999. and v[1] != -9999.:
        plt.ylim(v[0],v[1])
    plt.tight_layout()
    if filename != "":
        plt.savefig(filename,dpi=250)
        plt.close()
               
def x_cross_section(fignum, filename, figsize, title, grid, lith, y_plot,v):
    plt.figure(fignum, figsize = figsize)
    x_plot = grid.x_of_node[grid.y_of_node == y_plot]
    eta = grid.at_node['topographic__elevation'][grid.y_of_node == y_plot]
    
    num_layers = lith.z_top.shape[0]
    layers = np.zeros((num_layers-1,len(eta)))
    for k in range(0,num_layers-1):
        layers[k,:] = lith.z_top[k,:][grid.y_of_node == y_plot]
        plt.plot(x_plot,eta-layers[k,:])
        
    plt.plot(x_plot,eta,color='k')
    plt.title(title)
    plt.xlabel('x [meters]')
    plt.ylabel(r'$\eta$ [meters]')
    if v[0] == -9999. and v[1] != -9999.:
        plt.ylim(0,v[1])
    elif v[0] != -9999. and v[1] == -9999.:
        plt.ylim(v[0],)
    elif v[0] != -9999. and v[1] != -9999.:
        plt.ylim(v[0],v[1])
    plt.tight_layout()
    if filename != "":
        plt.savefig(filename,dpi=250)
        plt.close()
        
def y_cross_section(fignum, filename, figsize, title, grid, lith, x_plot,v):
    plt.figure(fignum, figsize = figsize)
    y_plot = grid.x_of_node[grid.y_of_node == x_plot]
    eta = grid.at_node['topographic__elevation'][grid.x_of_node == x_plot]
    
    num_layers = lith.z_top.shape[0]
    layers = np.zeros((num_layers-1,len(eta)))
    for k in range(0,num_layers-1):
        layers[k,:] = lith.z_top[k,:][grid.x_of_node == x_plot]
        plt.plot(y_plot,eta-layers[k,:])
        
    plt.plot(y_plot,eta,color='k')
    plt.title(title)
    plt.xlabel('y [meters]')
    plt.ylabel(r'$\eta$ [meters]')
    if v[0] == -9999. and v[1] != -9999.:
        plt.ylim(0,v[1])
    elif v[0] != -9999. and v[1] == -9999.:
        plt.ylim(v[0],)
    elif v[0] != -9999. and v[1] != -9999.:
        plt.ylim(v[0],v[1])
    plt.tight_layout()
    if filename != "":
        plt.savefig(filename,dpi=250)
        plt.close()

def add_folds(grid,field,slope,wavelength,amplitude,horizontal_line_boolean,vertical_line_boolean):
    if horizontal_line_boolean == True and vertical_line_boolean == True:
        sys.exit("Cannot have horizontal_line_boolean == True and vertical_line_boolean == True.")   
    elif horizontal_line_boolean == True:
        for i in range(0,grid.shape[0]):
            for j in range(0,grid.shape[1]):
                y = float(i) * grid.dy
                grid.at_node[field].reshape(grid.shape)[i,j] = amplitude * (1. - np.cos(2.0 * np.pi * y / wavelength))
    elif vertical_line_boolean == True:  
        for i in range(0,grid.shape[0]):
            for j in range(0,grid.shape[1]):
                x = float(j) * grid.dx
                grid.at_node[field].reshape(grid.shape)[i,j] = amplitude * (1. - np.cos(2.0 * np.pi * x / wavelength))
    else:
        if slope == 0.0:
            sys.exit("Slope cannot be zero. Try using horizontal_line_boolean == True or vertical_line_boolean == True instead.") 
        else:
            #origin at the center
            for i in range(0,grid.shape[0]):
                for j in range(0,grid.shape[1]):
                    x = float(j) * grid.dx
                    y = float(i) * grid.dy
                    x_o = (slope ** 2.0 * grid.extent[1] / 2.0 - slope * grid.extent[0] / 2.0 + slope * y + x) / (1. + slope ** 2.)
                    y_o = slope * (x_o - grid.extent[1] / 2.0) + grid.extent[0] / 2.0
                    dist = np.sqrt((x-x_o)**2.0 + (y-y_o)**2.0)
                    grid.at_node[field].reshape(grid.shape)[i,j] = amplitude * (1. - np.cos(2.0 * np.pi * dist / wavelength))

In [2]:
########################################################
###################ONLY MODIFY BELOW####################
########################################################

###Physical Parameters###
K = 0.0001 #1/yr, erodibility
D = 0.01 #m^2/yr, hillslope diffusion coefficient
U = 0.001 #m/yr #uplift rate
S = 0.0005 #m/yr soluability rate

###Numerical Parameters###
T = 500000. #years, total run time
dt = 100. #years, timestep
dt_plot = 10000. #years, plotting frequency
rows = 100 #y_cells
cols = 100 #x cells
dx = 50. #meters

###Save Parameters###
figsize = (5,4)
save_folder = "run_3" #name of the folder that the images will save to, if blank (i.e., "") no files will save 
plot_topography = True
max_elevation = 400.0 #max elevation to scale plots, use -9999 if you want to be automated
plot_drainage_area = True
plot_rock_type = True
plot_dissolution_rate = True
plot_profile = True
plot_x_cross_section = True
plot_y_cross_section = True
min_lithology_elevation = -100. #max elevation to scale plots, use -9999 if you want to be automated

#boundary conditions
closed_bc_top = True
closed_bc_bottom = False
closed_bc_right = True
closed_bc_left = True

#makes the grid - don't modify this line
grid = RasterModelGrid((rows,cols),dx)

#Lithology - Dissolution
# thicknesses = [200.,200.,200.]
# ids = [1, 2, 1]
# attrs = {'K_sp': {1: K, 2: K},'diss': {1: 0.0, 2: S}}

#Lithology - Folds
fold = grid.add_zeros('fold', at = 'node')
fold_thickness = 200
add_folds(grid,'fold',0.0,dx*rows/2.0,100.,True,False) 
thicknesses = [fold,\
               np.ones(grid.number_of_nodes) * fold_thickness,\
               np.ones(grid.number_of_nodes) * 100000.]
ids = [1, 2, 1]
attrs = {'K_sp': {1: K, 2: K*0.5},'diss': {1: 0.0, 2: 0.0}}

########################################################
###################ONLY MODIFY ABOVE####################
########################################################
start_time = time.time()

#set random seed, so randomization is the same every time
np.random.seed(1234)

#x and y
x = np.linspace(0.,dx*(cols-1),cols)
y = np.linspace(0.,dx*(rows-1),rows)

#make the folder if it doesn't exist
main_folder = os.getcwd()
save_folder = main_folder + '/' + save_folder
if os.path.exists(save_folder) == True:
    shutil.rmtree(save_folder)
    time.sleep(5)
os.mkdir(save_folder)

#make topographic elevation field with all zeros
grid.add_zeros('topographic__elevation', at = 'node')

#This sets the boundary conditions to open (False) - Close (True)
#(right edge, top edge, left edge, bottom edge)
grid.set_closed_boundaries_at_grid_edges(closed_bc_right,closed_bc_top,closed_bc_left,closed_bc_bottom)

###NOTE: No need to modify anything below this###
#reset the landscape at the beginning of the run
reset_landscape_random(grid)

#set up lithlogy
for i in range(0,len(ids)):
    if i == 0:
        thicknesses[i] = thicknesses[i] + grid.at_node['topographic__elevation']
    else:
        thicknesses[i] = thicknesses[i] + np.zeros(grid.number_of_nodes)
        
lith = Lithology(grid, thicknesses, ids, attrs, dz_advection = U * dt)
#set up flow accumulator, finds the drainage area
flow = FlowAccumulator(grid,flow_director="D8",depression_finder = 'DepressionFinderAndRouter')
#set up fuvial erosion
fluvial = FastscapeEroder(grid, K_sp = grid.at_node['K_sp'],m_sp=0.5,n_sp=1.0)
#set up the hillslope diffusion
diffuse = LinearDiffuser(grid, linear_diffusivity=D, deposit = False)

K_values = list(attrs['K_sp'].values())
diss_values = list(attrs['diss'].values())

#set initial figure number
fignum = 1
total_plots = int(T/dt_plot + 1)
for t in range(0,int(T/dt)+1):
    #run flow accumulator
    flow.run_one_step()
    #run fluvial erosion, this requires flow accumulator to run first
    fluvial.run_one_step(dt=dt)
    #run hillslope diffusion
    diffuse.run_one_step(dt=dt)
    #dissolve landscape
    grid.at_node['topographic__elevation'][grid.core_nodes] -= grid.at_node['diss'][grid.core_nodes]  * dt
    #uplift landscape
    grid.at_node['topographic__elevation'][grid.core_nodes] += U * dt
    #update lithlogy
    lith.run_one_step()
    if t%(int(dt_plot/dt))==0:
        year = round(t * dt)
        print ('plotting at '+str(year)+' years...')
        if plot_topography == True:
            if max_elevation == -9999:
                max_elevation = np.max(grid.at_node['topographic__elevation'][grid.core_nodes])
            simple_plot(fignum,save_folder+'/topography_'+str(year)+'yrs.png',\
                        figsize,'T = ' + str(round(t*dt/1000)) +' kyrs',grid,'topographic__elevation','elevation [m]','terrain',False,[0,max_elevation])
        if plot_drainage_area == True:
            simple_plot(fignum+total_plots,save_folder+'/drainage_area_'+str(year)+'yrs.png',\
                        figsize,'T = ' + str(round(t*dt/1000)) +' kyrs',grid,'drainage_area',r'$log_10$(drainage area) [-]','viridis',True,[dx*dx,dx*dx*rows*cols])
        if plot_rock_type == True:
            simple_plot(fignum+total_plots*2,save_folder+'/rock_'+str(year)+'yrs.png',\
                        figsize,'T = ' + str(round(t*dt/1000)) +' kyrs',grid,'K_sp','K [1/yr]','coolwarm',False,[np.min(K_values),np.max(K_values)])
        if plot_dissolution_rate == True:
            simple_plot(fignum+total_plots*3,save_folder+'/diss_'+str(year)+'yrs.png',\
                        figsize,'T = ' + str(round(t*dt/1000)) +' kyrs',grid,'diss','S [m/yr]','Blues',False,[np.min(diss_values),np.max(diss_values)])
        if plot_profile == True:
            profile_plot(fignum+total_plots*4,save_folder+'/profile_'+str(year)+'yrs.png',\
                        figsize,'T = ' + str(round(t*dt/1000)) +' kyrs',grid,lith,[min_lithology_elevation,max_elevation])
        if plot_x_cross_section == True:
            x_cross_section(fignum+total_plots*5,save_folder+'/x_cross_section_'+str(year)+'yrs.png',\
                        figsize,'T = ' + str(round(t*dt/1000)) +' kyrs',grid,lith,y[int(rows/2)],[min_lithology_elevation,max_elevation])
        if plot_y_cross_section == True:
            y_cross_section(fignum+total_plots*6,save_folder+'/y_cross_section_'+str(year)+'yrs.png',\
                        figsize,'T = ' + str(round(t*dt/1000)) +' kyrs',grid,lith,x[int(cols/2)],[min_lithology_elevation,max_elevation])
        fignum+=1
        
end_time = time.time()
print ('code took ' + str(round((end_time-start_time)/60.0,1)) + ' minutes to run')

plotting at 0 years...
plotting at 10000 years...
plotting at 20000 years...
plotting at 30000 years...
plotting at 40000 years...
plotting at 50000 years...
plotting at 60000 years...
plotting at 70000 years...
plotting at 80000 years...
plotting at 90000 years...
plotting at 100000 years...
plotting at 110000 years...
plotting at 120000 years...
plotting at 130000 years...
plotting at 140000 years...
plotting at 150000 years...
plotting at 160000 years...
plotting at 170000 years...
plotting at 180000 years...
plotting at 190000 years...
plotting at 200000 years...
plotting at 210000 years...
plotting at 220000 years...
plotting at 230000 years...
plotting at 240000 years...
plotting at 250000 years...
plotting at 260000 years...
plotting at 270000 years...
plotting at 280000 years...
plotting at 290000 years...
plotting at 300000 years...
plotting at 310000 years...
plotting at 320000 years...
plotting at 330000 years...
plotting at 340000 years...
plotting at 350000 years...
plotti